# Secção de Código

In [167]:
# ATENÇÃO À UTILIZAÇÃO DESTA FUNÇÃO
# DEMASIADAS QUERIES SEGUIDAS PODEM LEVAR A BLOQUEIO POR PARTE DO SERVIDOR
# EM PRINCÍPIO O MÓDULO ENTREZ ESTÁ PREPARADO PARA LIDAR COM ISSO MAS CONVÉM TER SEMPRE CUIDADO

def pesquisa_ncbi(email, term, db = 'pubmed', retmax = 10, rettype = 'abstract', retmode = 'text', display = 'Y' , save = 'N', ext='txt'):
    '''
    NOTA - PARA USAR RETMODE XML TÊM DE SER DECLARADA UMA NOVA VARIÁVEL POIS A FUNÇÃO AINDA NÃO CRIA FICHEIROS XML
    
    Função para a pesquisa de bibliografia na base de dados NCBI utilizando BioPython
    Esta função contempla um conjunto de parâmetros para seleção de base de dados e outputs desejado adaptando-se
    ao objetivo da pesquisa. 

    Por defeito a função pesquisa a base de dados pubmed e capta os 10 primeiros resultados para uma pesquisa
    generalizada. 

    Tem a opção de gerar um ficheiro de texto com os resultados da pesquisa de artigos e respetivos abstracts.

    No entanto a função pode ser especializada para uma pesquisa em base de dados de genes para captar ficheiros
    GenBank e FASTA para posterior análise com as funções específicas que estamos a desenvolver.

    Parâmetros
    ----------

    email : str
        email a ser utilizado pelo módulo Entrez para  pesquisa (e.g. exempl@mail.com)

    term : str
        termo a ser utilizado na pesquisa (e.g. 'gene-123 functions')

    db  : str
        base de dados onde irá ser realizada a pesquisa. Por defeito é utilizada a 'pubmed'
        Para efeitos deste trabalho iremos também utilizar: 'gene', 'nuccore' e 'protein'
        Outras opções disponíveis podem ser encontradas aqui: https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/?report=objectonly

    retmax : int
        define o número de resultados que queremos retirar para o output final (10 por defeito)

    rettype : str
        define o tipo de resultado que pretendemos retirar da base de dados ('abstract' por defeito)
        pode ser mudado para 'gb' para base de dados 'gene' ou 'fasta' para base de dados de nucleotidos ou proteina (sequencias)
        
    retmode : str
        define o retmode para a captura de resultados da pesquisa na base de dados. Por defeito 
        o retmode é text mas pode ser adaptado para 'xml' quando é pretendido obter um ficheiro xml (e.g base de dados nucleotide, ou genbank)

    display : str
        define se é pretendido o output da função legível (útil para quando se quer refinar a pesquisa antes/sem gravar o ficheiro)
        por defeito apresenta o resultado em formato legível
    
    save : str
        define se é pretendido gravar ou não o ficheiro de resultados na pasta 'lit-search-output'
        por defeito este parâmetro está inactivo

            
    
    '''
    # TODO criar ficheiros XML
    


    # Importação de módulos
    from Bio import Entrez
    import os

    # Definimos o email para os fins de pesquisa
    Entrez.email = email
        
    # Secção egquery que pode ser skipable

    handle = Entrez.egquery(term=term)
    egq_res = Entrez.read(handle)

    for _ in egq_res['eGQueryResult']:
        if db in _.values():
            if retmax > 1:
                print('Encontrados {} resultados em {}. Irão ser processados {} resultados.\n'.format(_['Count'],db,retmax))
            else:
                print('Encontrados {} resultados em {}. Irá ser processado 1 resultado.\n'.format(_['Count'],db))

    
    if rettype == 'fasta' or rettype == 'gb': retmax = 1

    # Sacar as IDs
    handle      = Entrez.esearch(db=db,term=term, retmode=retmode, retmax=retmax)
    esearch_res = Entrez.read(handle)

    lista_ids = esearch_res['IdList']
    # print('Top 10 artigos ->',lista_ids) # Converter para uma lista de títulos mais à frente'''
    
    

    # Transfere a informação
    handle = Entrez.efetch(db=db,id=','.join(lista_ids),rettype=rettype,retmode=retmode)
    fetch_res = handle.read()
    
    # Usar com print para já, para não estar a bagunçar demasiado.
    # Perceber como se pode implementar a escrita de ficheiros para ser mais fácil de mastigar os resultados.

    # NÃO FUNCIONA NO GITHUB / LIVECODING
    # ADICIONAR PARAMETRO save = 'Y' SE SE PRETENDER GUARDAR FICHEIRO 
    
    if display.upper() == 'Y':
        print(fetch_res,sep='\n')


    # Verifica a condição de gravação de output como ficheiro de texto
    
    if retmode == 'xml': save = 'n'
    
    if save.upper() == 'Y':  

        # Definimos a extensão em função da base de dados e retmode
        if rettype == 'gb':
            ext = '.gbk'
        
        elif rettype == 'fasta':
            ext = '.fasta'
        
        elif retmode == 'xml':
            ext = '.xml'

        else: ext = '.txt'

        filename = term+ext

        # Verifica se existe pasta para output e cria-a caso não exista
        cwd = os.getcwd()
        outputdir = os.path.join(cwd,'output')
        if not os.path.exists(outputdir):
            os.mkdir(outputdir)


        # Muda o working directory para a pasta de output para gerar o ficheiro
        os.chdir(outputdir)

        if os.path.isfile(filename): print('Ficheiro já existe. Não será criado novo ficheiro.')



        # Gera o ficheiro           
        with open(term+ext , 'w', encoding='utf-8') as _:
            _.write(fetch_res)

        # Retorna ao working directory anterior
        os.chdir(cwd)

        print('Ficheiro gravado.')


    handle.close()

    print('Concluído.')

    return fetch_res

In [157]:
def parsing(nome_ficheiro):

    from Bio import SeqIO
    
    record = SeqIO.read(nome_ficheiro, "genbank")

    print("ID do gene:", record.id)

    print("Nome do gene:", record.name)

    print("Descrição do gene", record.description)

    print("Comprimento da sequência:", len(record.seq), "bp")

    return

In [158]:
def anot(nome_ficheiro):
    
    from Bio import SeqIO
    
    record = SeqIO.read(nome_ficheiro, "genbank")

    print("Quantidade de anotações:", len(record.annotations))

    print()
    
    print("Lista de anotações:")
    
    for anotacao in record.annotations:
        print(anotacao, "->", record.annotations[anotacao])

    return   


In [159]:
def features_qualifiers(nome_ficheiro):
    
    from Bio import SeqIO
    
    record = SeqIO.read(nome_ficheiro, "genbank")
    
    print("Quantidade de features:", len(record.features))

    for feature in record.features:
        print(feature)
    return


# Secção de resultados

## Literatura

In [160]:
email = 'pg21019@alunos.uminho.pt'
gene = 'hla-dqa2'
termos = [
    
    'hla-dqa1 hla-dqa2 hla-dqb1',
    'hla-dqa2 food allergy',
    'hla-dqa2 cancer'
]

In [ ]:
# Pesquisa generalizada - top 10 resultaods

pesquisa_ncbi(email,gene)

In [ ]:
pesquisa_ncbi(email,'HLA-DQA2 homo sapiens',db='protein',retmax=90)

In [ ]:
# Pesquisa para os restantes termos
from time import sleep

for termo in termos:
    pesquisa_ncbi(email,termo,save='y',display='n')
    sleep(5)


In [ ]:
# Identificação dos dados para pesquisa avançada
# Geração de ficheiros para processamento nas outras funções
# TODO alterar para save = 'y' no final
nucleotide = pesquisa_ncbi(email,gene,db='nuccore',save='n',retmax=1,display='n')          # Descobrir o identificador do gene
proteina   = pesquisa_ncbi(email,gene,db='protein',save='n',retmax=1,display='n') # Descobrir o identificador da proteína

In [ ]:
id_nucleotide = nucleotide.split()[1]
id_proteina = proteina.split()[1]

In [ ]:
print(id_nucleotide,id_proteina)

In [ ]:
# Transferimos os ficheiros FASTA e GBK para uso futuro
pesquisa_ncbi(email,id_nucleotide,db='nuccore',retmax=1,rettype='gb',save='y',display='n')

In [ ]:
pesquisa_ncbi(email,id_proteina,db='protein',retmax=1,rettype='fasta',save='y',display='n')

In [ ]:
fich_gbk   = 'gene_search() output\\' + id_nucleotide + '.gbk'
fich_prot_fasta = 'gene_search() output\\' + id_proteina + '.fasta'

In [ ]:
parsing(fich_gbk)

In [ ]:
anot(fich_gbk)

In [ ]:
features_qualifiers(fich_gbk)

## Análise BLAST

## Análise da proteína

In [161]:
from Bio import SeqIO
fich_prot_fasta = 'output\\NP_064440.fasta'

# Lemos o ficheiro .fasta
seq_proteina = SeqIO.read(open(fich_prot_fasta),format='fasta')

sp = seq_proteina.seq # Isolamos a sequência

In [162]:
print(seq_proteina)

ID: NP_064440.1
Name: NP_064440.1
Description: NP_064440.1 HLA class II histocompatibility antigen, DQ alpha 2 chain precursor [Homo sapiens]
Number of features: 0
Seq('MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQSHGPSGQYTHEFDG...GLL')


In [163]:
# Procedemos à contagem de aminoácidos:
# Primeiro para a sequência traduzida e depois para o conjunto de sequências após a remoção de codões stop

sp = seq_proteina.seq

def conta_aa(seq_prot):
    # TODO colocar um dicionário bonitinho
    from collections import Counter
    contagem_aa = dict(Counter(seq_prot))
    
    return contagem_aa

# conta_aa(sp)


In [164]:
swiss_id = 'P01906' # Encontra-se o ID no ficheiro FASTA 

def swiss_prot_scan(swiss_id):
    from Bio import SeqIO
    from Bio import ExPASy
    # Efetuamos uma pesquisa na base de dados SwissProt 
    handle = ExPASy.get_sprot_raw(swiss_id) # Encontramos o ID ao ler o ficheiro fasta
    sr = SeqIO.read(handle, "swiss")
    print(
        f'ID {sr.id}',
        f'Sequência: {sr.seq}',
        f'Tamanho da sequência: {len(sr.seq)} bp',
        f'Nome: {sr.name}',
        f'Descrição: {sr.description}',
        f'Taxonomia: {sr.annotations["taxonomy"]}',
        f'Organismo: {sr.annotations["organism"]}',
        f'Keywords: {sr.annotations["keywords"]}',
        sep = '\n')

swiss_prot_scan('P01906')

ID P01906
Sequência: MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQSHGPSGQYTHEFDGDEEFYVDLETKETVWQLPMFSKFISFDPQSALRNMAVGKHTLEFMMRQSNSTAATNEVPEVTVFSKFPVTLGQPNTLICLVDNIFPPVVNITWLSNGHSVTEGVSETSFLSKSDHSFFKISYLTFLPSADEIYDCKVEHWGLDEPLLKHWEPEIPAPMSELTETLVCALGLSVGLMGIVVGTVFIIQGLRSVGASRHQGLL
Tamanho da sequência: 255 bp
Nome: DQA2_HUMAN
Descrição: RecName: Full=HLA class II histocompatibility antigen, DQ alpha 2 chain; AltName: Full=DX alpha chain; AltName: Full=HLA class II histocompatibility antigen, DQ(6) alpha chain; AltName: Full=HLA-DQA1; AltName: Full=MHC class II DQA2; Flags: Precursor;
Taxonomia: ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
Organismo: Homo sapiens (Human)
Keywords: ['Adaptive immunity', 'Cell membrane', 'Disulfide bond', 'Endoplasmic reticulum', 'Endosome', 'Glycoprotein', 'Golgi apparatus', 'Immunity', 'Lysosome', 'Membrane', 'MHC II', '

In [166]:
def analise_blast(email, sequencia_proteina, filename, lim = None):
    import os
    from Bio.Blast import NCBIWWW
    
    NCBIWWW.email = email

    
    
    # Verifica se já existe um ficheiro XML para os resultados do BLAST. Ignora a pesquisa caso já exista.
    # Para poupar tempo.
    # Se o ficheiro já existir executa o parsing do ficheiro.

    if '.xml' not in filename:
        filename = filename + '.xml'

    if os.path.isfile(filename): 
        
        blast_parse(filename,lim)

    else: 
    # Executa o BLAST da Proteína
        handle = NCBIWWW.qblast('blastp','swissprot',sequencia_proteina.seq)

        with open(filename, 'w') as _:
            _.write(handle.read())
        
        handle.close()

analise_blast('pg21019@alunos.uminho.pt', seq_proteina,'blast_results.xml')

****Alinhamento 1 ****
Sequência: sp|P01906.2| RecName: Full=HLA class II histocompatibility antigen, DQ alpha 2 chain; AltName: Full=DX alpha chain; AltName: Full=HLA class II histocompatibility antigen, DQ(6) alpha chain; AltName: Full=HLA-DQA1; AltName: Full=MHC class II DQA2; Flags: Precursor [Homo sapiens]
Tamanho da Sequência: 255
e-value: 0.0
MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQ...
MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQ...
MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQ...

****Alinhamento 2 ****
Sequência: sp|P01909.1| RecName: Full=HLA class II histocompatibility antigen, DQ alpha 1 chain; AltName: Full=DC-1 alpha chain; AltName: Full=DC-alpha; AltName: Full=HLA-DCA; AltName: Full=MHC class II DQA1; Flags: Precursor [Homo sapiens]
Tamanho da Sequência: 254
e-value: 4.22054e-162
MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQ...
MILNKAL+LGALALT VMSPCGGEDIVADHVASYGVN YQ...
MILNKALMLGALALTTVMSPCGGEDIVADHVASYGVNLYQ...

****Alinhamento 3 ****
Sequência: sp|P15981.1| RecName: Full=SLA class

In [ ]:
filename = 'blast_results.xml'

from Bio.Blast import NCBIXML
handle = open('blast_results.xml')
blast_records = NCBIXML.parse(handle)
brs = list(blast_records)

In [165]:
filename = 'blast_results.xml'

def blast_parse(filename, lim = None):
    
    from Bio.Blast import NCBIXML

    handle = open(filename)
    blast_res = NCBIXML.parse(handle)
    blast_records = list(blast_res)

    num = 1
    for record in blast_records:

        if lim == None: lim = len(record.alignments)    

        for aligment in record.alignments[0:lim]:
            #print(aligment)
            
            for hsp in aligment.hsps:
                #print(hsp)
                
                if hsp.expect < 0.05:
                    print('****Alinhamento {} ****'.format(num))
                    print('Sequência:',aligment.title)
                    print('Tamanho da Sequência:',aligment.length)
                    print('e-value:',hsp.expect)
                    print(hsp.query[0:40]+'...')
                    print(hsp.match[0:40]+'...')
                    print(hsp.sbjct[0:40]+'...')
                    print()
                    num += 1    

blast_parse(filename,1)

****Alinhamento 1 ****
Sequência: sp|P01906.2| RecName: Full=HLA class II histocompatibility antigen, DQ alpha 2 chain; AltName: Full=DX alpha chain; AltName: Full=HLA class II histocompatibility antigen, DQ(6) alpha chain; AltName: Full=HLA-DQA1; AltName: Full=MHC class II DQA2; Flags: Precursor [Homo sapiens]
Tamanho da Sequência: 255
e-value: 0.0
MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQ...
MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQ...
MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQ...



In [ ]:
from Bio import SearchIO
bresult = SearchIO.read('blast_results.xml', 'blast-xml')

In [ ]:
# Top 5 HSPs
for _ in range(0,5):
    print(bresult[_][0])
    print()